# TP3 Ex1 Grupo 24
Gabriel Antunes a101101
<br>
Guilherme Pinho a105533

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  a,b > 0  e devolve inteiros r, s, t tais que a * s + b * t = r  e  r = gcd(a,b). 

  Para além das variáveis r,s,t o código requer 3 variáveis adicionais r',s',t que representam os valores de r,s,t no “próximo estado”.
    
    INPUT  a, b
    assume  a > 0 and b > 0
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
    OUTPUT r, s, t
    


  1. Construa um SFOTS usando BitVector’s de tamanho n que descreva o comportamento deste programa.  Considere estado de erro quando r=0 ou alguma das variáveis atinge o “overflow”.

In [3]:
from pysmt.shortcuts import * 
from pysmt.typing import BVType

import itertools 

bit_size = 5 # n do enunciado
initA = 24
initB = 36

def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i), types.BVType(bit_size))
    return state

def init(s):
    return And(
        # Equals(s['a'], BV(initA, bit_size)),  # Valores teste
        # Equals(s['b'], BV(initB, bit_size)),
        BVUGT(s['a'], BV(0, bit_size)),         # a > 0 
        BVUGT(s['b'], BV(0, bit_size)),         # b > 0 
        Equals(s['r'], s['a']),                 # r  == a
        Equals(s['rr'], s['b']),                # r' == b
        Equals(s['s'], BV(1, bit_size)),        # s  == 1
        Equals(s['ss'], BV(0, bit_size)),       # s' == 0
        Equals(s['t'], BV(0, bit_size)),        # t  == 0
        Equals(s['tt'], BV(1, bit_size))        # t' == 1
    )

def error(s, bit_size):
    max_value = (1 << bit_size) - 1     # Valor máximo do BitVector
    max_bv = BV(max_value, bit_size)    # Representação como BitVector
    
    return Or(
        Equals(s['r'], BV(0, bit_size)),    # Caso r atinja o valor 0
        BVUGT(s['r'], max_bv),              # Caso r tenha overflow
        BVUGT(s['rr'], max_bv),   
        BVUGT(s['s'], max_bv),     
        BVUGT(s['t'], max_bv)      
    )

def trans(curr, prox):

    return And(
        BVUGT(curr['rr'], BV(0, bit_size)),                                  # rr > 0
        Equals(prox['q'], BVUDiv(curr['r'], curr['rr'])),                    # q = r // rr
        Equals(prox['r'], curr['rr']),                                       # r = rr
        Equals(prox['rr'], BVSub(curr['r'], BVMul(prox['q'], curr['rr']))),  # rr = r - q * rr
        Equals(prox['s'], curr['ss']),                                       # s = ss
        Equals(prox['ss'], BVSub(curr['s'], BVMul(prox['q'], curr['ss']))),  # ss = s - q * ss
        Equals(prox['t'], curr['tt']),                                       # t = tt
        Equals(prox['tt'], BVSub(curr['t'], BVMul(prox['q'], curr['tt'])))   # tt = t - q * tt
    )

def final_condition(s, bit_size):

    return And(
        Equals(BVURem(s['a'], s['r']), (BV(0, bit_size))),   # r divide a
        Equals(BVURem(s['b'], s['r']), (BV(0, bit_size))),   # r divide b
        BVUGT(s['r'], BV(0, bit_size)),
        Equals(s['rr'], BV(0, bit_size))
    )

def genTrace(vars, init, trans, final_condition, error, n):
    
    with Solver(name="z3") as solver:
        X = [genState(vars, 'X', i) for i in range(n+1)]     # cria n+1 estados (com etiqueta X)
        
        I = init(X[0]) 
        Tks = [trans(X[i], X[i+1]) for i in range(n)] 
        F = final_condition(X[-1], bit_size)
        
        if solver.solve([I, And(Tks), F]): 
            for i in range(n+1):
                print(f"Estado {i}:")
                for v in X[i]:
                    print(f"  {v} = {solver.get_value(X[i][v])}")
        else:
            print("Nenhum traço válido encontrado.")
            
vars = ['r', 'rr', 's', 'ss', 't', 'tt', 'q', 'a', 'b']

genTrace(vars, init, trans, final_condition, error, 7) 

Estado 0:
  r = 19_5
  rr = 30_5
  s = 1_5
  ss = 0_5
  t = 0_5
  tt = 1_5
  q = 0_5
  a = 19_5
  b = 30_5
Estado 1:
  r = 30_5
  rr = 19_5
  s = 0_5
  ss = 1_5
  t = 1_5
  tt = 0_5
  q = 0_5
  a = 0_5
  b = 0_5
Estado 2:
  r = 19_5
  rr = 11_5
  s = 1_5
  ss = 31_5
  t = 0_5
  tt = 1_5
  q = 1_5
  a = 0_5
  b = 0_5
Estado 3:
  r = 11_5
  rr = 8_5
  s = 31_5
  ss = 2_5
  t = 1_5
  tt = 31_5
  q = 1_5
  a = 0_5
  b = 0_5
Estado 4:
  r = 8_5
  rr = 3_5
  s = 2_5
  ss = 29_5
  t = 31_5
  tt = 2_5
  q = 1_5
  a = 0_5
  b = 0_5
Estado 5:
  r = 3_5
  rr = 2_5
  s = 29_5
  ss = 8_5
  t = 2_5
  tt = 27_5
  q = 2_5
  a = 0_5
  b = 0_5
Estado 6:
  r = 2_5
  rr = 1_5
  s = 8_5
  ss = 21_5
  t = 27_5
  tt = 7_5
  q = 1_5
  a = 0_5
  b = 0_5
Estado 7:
  r = 1_5
  rr = 0_5
  s = 21_5
  ss = 30_5
  t = 7_5
  tt = 13_5
  q = 2_5
  a = 0_5
  b = 0_5


2. Prove, usando a metodologia dos invariantes e interpolantes, que o modelo nunca atinge o estado de erro.

In [4]:
def invert(trans):
    return lambda curr, prox: trans(prox, curr)

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

def model_checking(vars,init,trans,error,N,M):
    with Solver(name="z3") as solver:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        transt = invert(trans)
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        # Step 1 implícito na ordem de 'order' e nas definições de Rn, Um.
        for (n,m) in order:      
            # Step 2. 
            I = init(X[0])
            Tn = And([trans(X[i], X[i+1]) for i in range(n)])
            Fn = final_condition(X[n], bit_size)
            Rn = And(I, Tn, Fn)
            
            E = error(Y[0], bit_size)
            Bm = And([transt(Y[i], Y[i+1]) for i in range(m)])
            Um = And(E, Bm)
            
            Vnm = And(Rn, same(X[n], Y[m]), Um)
            if solver.solve([Vnm]):
                print("> O sistema é inseguro.")
                return
            else:
                # Step 3. 
                A = And(Rn, same(X[n], Y[m]))
                B = Um
                C = binary_interpolant(A, B)
                
                # Salvaguardar cálculo bem-sucedido do interpolante.
                if C is None:
                    print(f"> Não foi possível calcular o interpolante em n={n}, m={m}.")
                    return

                
                # Step 4. 
                C0 = rename(C, X[0])
                T = trans(X[0], X[1])
                C1 = rename(C, X[1])
                F1 = final_condition(X[1], bit_size)

                if not solver.solve([C0, T, F1, Not(C1)]):
                    # C é invariante de T.
                    print("> O sistema é seguro.")
                    return 
                else:
                    # Step 5.1.
                    S = rename(C, X[n])
                    while True:
                        # Step 5.2.
                        T = trans(X[n], Y[m])
                        A = And(S, T)
                        if solver.solve([A, Um]):
                            print("> Não foi encontrado majorante.")
                            break 
                        else:
                            # Step 5.3.
                            C = binary_interpolant(A, Um)
                            Cn = rename(C, X[n])
                            if not solver.solve([Cn, Not(S)]):
                                # Step 5.4.
                                # C(Xn) -> S é tautologia.
                                print("> O sistema é seguro.")
                                return
                            else:
                                S = Or(S, Cn)
                                
    print("> Não foi provada a segurança ou insegurança do sistema.")
                            
model_checking(vars, init, trans, error, 7, 7) 

NoSolverAvailableError: No Interpolator is available